# GHI Sandbox – Demo Notebook
Global HashCost Index Initiative  
Version : Sandbox v0.2.0

Ce notebook montre comment :

- se connecter à l'API sandbox GHI,
- récupérer un snapshot global,
- explorer l'historique,
- analyser les régions,
- visualiser les coûts de production synthétiques,
- utiliser le client Python officiel.

## 1. Prérequis

Avant d'exécuter ce notebook :

1. Cloner et installer `ghi-engine` (sandbox).
2. Lancer le serveur FastAPI :

```bash
cd ghi-engine
source venv/bin/activate
uvicorn app.main:app --reload
```

L'API doit répondre sur :

- `http://127.0.0.1:8000`
- base sandbox : `http://127.0.0.1:8000/v1/ghi`

In [ ]:
%pip install --quiet requests pandas matplotlib

In [ ]:
import os
import sys
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)

CLIENT_PATHS = [
    ".",
    "./python",
    "./devpack/python",
]

for p in CLIENT_PATHS:
    full = os.path.abspath(p)
    if full not in sys.path and os.path.exists(full):
        sys.path.insert(0, full)

try:
    from ghi_sandbox_client import GHISandboxClient, GHIClientConfig
    print("Client officiel GHI importé.")
except ImportError as e:
    print("Impossible d'importer GHISandboxClient.")
    print("Vérifie le chemin de ghi_sandbox_client.py.")
    raise e

config = GHIClientConfig(
    base_url="http://127.0.0.1:8000/v1/ghi",
    timeout=10,
)
client = GHISandboxClient(config)

In [ ]:
print("Test de connectivité à la sandbox GHI...")

snapshot = client.get_snapshot()
print("OK. Timestamp snapshot :", snapshot.get("timestamp"))
print("Coût moyen global (BTC) :", snapshot.get("ghi", {}).get("avg_cost_btc"))

In [ ]:
from pprint import pprint

print("=== SNAPSHOT GLOBAL ===")
pprint(snapshot)

print("\nRésumé rapide :")
ghi = snapshot.get("ghi", {})
print(f"Coût min (BTC) : {ghi.get('min_cost_btc')}")
print(f"Coût moyen (BTC) : {ghi.get('avg_cost_btc')}")
print(f"Coût max (BTC) : {ghi.get('max_cost_btc')}")
print(f"Difficulté : {snapshot.get('difficulty')}")
print(f"Hashrate total (TH/s) : {snapshot.get('hashrate_total_th')}")
print(f"Nombre de régions : {len(snapshot.get('regions', []))}")

In [ ]:
history = client.get_history()
len(history), history[:2]

In [ ]:
df_hist = pd.DataFrame(history)
df_hist["timestamp"] = pd.to_datetime(df_hist["timestamp"])

df_hist.head()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_hist["timestamp"], df_hist["min_cost_btc"], label="min_cost_btc")
plt.plot(df_hist["timestamp"], df_hist["avg_cost_btc"], label="avg_cost_btc")
plt.plot(df_hist["timestamp"], df_hist["max_cost_btc"], label="max_cost_btc")
plt.xlabel("Date")
plt.ylabel("Coût synthétique (BTC)")
plt.title("GHI Sandbox – Coût min / moyen / max (BTC)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
regions = client.list_regions()
len(regions)

In [ ]:
df_regions = pd.DataFrame(regions)
df_regions

In [ ]:
plt.figure(figsize=(8, 5))
plt.bar(df_regions["name"], df_regions["hashrate_pct"])
plt.ylabel("Part de hashrate (0–1)")
plt.title("GHI Sandbox – Part de hashrate par région (synthétique)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
region_id = df_regions.iloc[0]["region_id"]
print("Region ID sélectionnée :", region_id)

region_detail = client.get_region(region_id)
from pprint import pprint
pprint(region_detail)

In [ ]:
def extract_region_costs(regions_data):
    rows = []
    for r in regions_data:
        cost = r.get("cost", {})
        rows.append(
            {
                "region_id": r.get("region_id"),
                "name": r.get("name"),
                "hashrate_pct": r.get("hashrate_pct"),
                "min_cost_btc": cost.get("min_cost_btc"),
                "avg_cost_btc": cost.get("avg_cost_btc"),
                "max_cost_btc": cost.get("max_cost_btc"),
            }
        )
    return pd.DataFrame(rows)

df_costs = extract_region_costs(regions)
df_costs

In [ ]:
plt.figure(figsize=(8, 5))
plt.bar(df_costs["name"], df_costs["avg_cost_btc"])
plt.ylabel("Coût moyen (BTC)")
plt.title("GHI Sandbox – Coût moyen (BTC) par région (synthétique)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
stats = client.get_stats()
from pprint import pprint
pprint(stats)

print("\nRésumé :")
print("Nombre de régions :", stats.get("regions_count"))
print("Hashrate total moyen (TH/s) :", stats.get("avg_hashrate_total_th"))
print("Coût moyen global (BTC) :", stats.get("avg_cost_btc"))
print("Notes :", stats.get("notes"))

In [ ]:
def print_quick_report():
    snap = client.get_snapshot()
    stats = client.get_stats()
    regions_local = client.list_regions()

    ghi = snap.get("ghi", {})
    print("=== GHI Sandbox – Quick Report ===")
    print("Timestamp :", snap.get("timestamp"))
    print(
        f"GHI min / avg / max (BTC) : "
        f"{ghi.get('min_cost_btc')} / {ghi.get('avg_cost_btc')} / {ghi.get('max_cost_btc')}"
    )
    print("Difficulty :", snap.get("difficulty"))
    print("Hashrate total (TH/s) :", snap.get("hashrate_total_th"))
    print("Regions count :", stats.get("regions_count"))
    print("Avg cost global (BTC) :", stats.get("avg_cost_btc"))
    print("Sample regions :", ", ".join(r.get("name") for r in regions_local[:3]))

print_quick_report()

## Conclusion

Ce notebook a montré comment :

- se connecter à la sandbox GHI,
- consommer les endpoints principaux (`/snapshot`, `/history`, `/regions`, `/stats`),
- transformer les réponses en DataFrames,
- visualiser des séries temporelles et des coûts régionaux,
- produire un mini-rapport synthétique.

La sandbox ne reflète pas des données réelles, mais la structure est identique à
celle de la future API de production (Phase 3).

Pour aller plus loin :

- intégrer ces appels dans un pipeline interne (ETL, data lake),
- créer des dashboards (Power BI, Tableau, Grafana),
- tester des scénarios de stress, des analyses de risque ou ESG à partir des
formats GHI.